In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Helper function to calculate SMAPE (user's formula, fractional form)
def smape_user_version(y_true, y_pred):
    smape = 0
    for i in range(len(y_true)):
        smape += abs(y_true[i] - y_pred[i]) / (abs(y_true[i]) + abs(y_pred[i]))
    smape /= len(y_true)
    return smape

# IQR Penalty Calculation
def calculate_iqr_penalty(actual, predictions):
    """Calculate IQR-based penalty."""
    iqr_actual = np.percentile(actual, 75) - np.percentile(actual, 25)
    iqr_pred = np.percentile(predictions, 75) - np.percentile(predictions, 25)
    
    if iqr_pred < iqr_actual:
        iqr_ratio = iqr_pred / iqr_actual
        penalty = (1 - iqr_ratio) * 0.3  # Penalty proportional to IQR difference
        return penalty
    else:
        return 0

# SARIMAX Recursive Forecasting with IQR Penalty
def sarimax_recursive_forecast_with_iqr(train_data, order=(1, 1, 1), steps=36, mc_iterations=10):
    predictions = []
    for iteration in range(mc_iterations):
        model = SARIMAX(train_data, order=order, seasonal_order=(0, 0, 0, 0))
        model_fit = model.fit(disp=False)
        forecasted_values = model_fit.forecast(steps=steps)
        predictions.append(forecasted_values)
    
    avg_forecasted_values = np.mean(predictions, axis=0)
    return avg_forecasted_values

# SMAPE with IQR Penalty Function using user's SMAPE formula
def sarimax_with_iqr_penalty_user_smape(selected_input_data, steps=36, order=(1, 1, 1), mc_iterations=10):
    # Splitting the dataset into target feature only
    target_series = selected_input_data  # Univariate data
    
    # Scaling the target data
    target_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_target = target_scaler.fit_transform(target_series.reshape(-1, 1)).flatten()
    
    # SARIMAX model recursive forecast with Monte Carlo iterations
    sarimax_predictions_scaled = sarimax_recursive_forecast_with_iqr(scaled_target, order=order, steps=steps, mc_iterations=mc_iterations)
    
    # Inverse transform predictions to the original target scale
    sarimax_predictions = target_scaler.inverse_transform(np.array(sarimax_predictions_scaled).reshape(-1, 1)).flatten()
    
    # Use the last `steps` data points for SMAPE comparison (if actual future values available)
    y_true = target_series[-steps:]
    y_pred = sarimax_predictions[:steps]
    
    # Calculate SMAPE using user's formula
    smape_score = smape_user_version(y_true, y_pred)
    
    # Apply IQR-based penalty
    penalty = calculate_iqr_penalty(y_true, y_pred)
    smape_with_penalty = smape_score + penalty
    
    return smape_with_penalty

# Running SARIMAX for univariate forecasting
def run_univariate_sarimax(target_column, data, steps=36, mc_iterations=10, n_iter=5):
    target_series = data[target_column].values  # Only target column for univariate

    best_smape = np.inf
    best_order = None
    best_predictions = None
    
    for i in range(n_iter):
        p = np.random.randint(0, 2)
        d = np.random.randint(0, 1)
        q = np.random.randint(0, 2)
        
        try:
            sarimax_predictions_scaled = sarimax_recursive_forecast_with_iqr(
                target_series, order=(p, d, q), steps=steps, mc_iterations=mc_iterations)
            
            y_true = target_series[-steps:]
            y_pred = sarimax_predictions_scaled[:steps]
            current_smape = smape_user_version(y_true, y_pred)
            
            if current_smape < best_smape:
                best_smape = current_smape
                best_order = (p, d, q)
                best_predictions = sarimax_predictions_scaled
                
        except Exception as e:
            print(f"Failed for order {(p, d, q)}: {str(e)}")
    
    return best_smape, best_order, best_predictions

def plot_forecast_with_years(actual_values, forecast_values, steps, start_year=2011, title="Forecast"):
    # Number of total points (historical + forecast)
    total_data_points = len(actual_values) + steps
    
    # Generate year labels for each year starting from the `start_year`
    years = [str(year) for year in range(start_year, start_year + (total_data_points // 12) + 1)]
    
    # Set ticks for each year (every 12 points corresponds to one year)
    ticks = np.arange(0, total_data_points, 12)
    
    # Create a combined array of actual and forecasted values for continuous line plotting
    combined_values = np.concatenate([actual_values, forecast_values])

    plt.figure(figsize=(12, 6))

    # Plot the combined data (actual + forecast)
    plt.plot(np.arange(total_data_points), combined_values, label='Actual + Forecast', color='blue')

    # Highlight the forecasted portion in a different style
    plt.plot(np.arange(len(actual_values), total_data_points), forecast_values, label='Forecasted Values (36 months)', color='red', linestyle='--')

    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel("Values")
    
    # Set year labels on the x-axis
    plt.xticks(ticks, years[:len(ticks)], rotation=45)
    
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{title}_ARIMA.png')
    plt.show()


# Example usage
data = pd.read_csv('../Input/InputData.csv')

attack_list = [
    'Mentions-IoT Device Attack',
    'Ransomware-ALL',
    'Cryptojacking-ALL',
    'Mentions-WannaCry Ransomware',
    'Mentions-Data Poisoning',
    'Mentions-Deepfake',
    'Mentions-Adversarial Attack'
]

for attack in attack_list:
    target_column = attack  # Example target column

    # Running the model for the target column
    best_smape, best_order, best_predictions = run_univariate_sarimax(target_column, data, mc_iterations=10, n_iter=60)

    print(f"Best SMAPE ({target_column}):", best_smape)
    print(f"Best Order ({target_column}):", best_order)

    # Plot the forecast with year labels
    target_series = data[target_column].values
    plot_forecast_with_years(target_series, best_predictions, steps=36, start_year=2011, title=f"{target_column} Forecast")
